In [34]:
import numpy as np
import warnings
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

class PCA:

    def __init__(self, principal_components_number:int=None, std_scale:bool=False, features_weights=None, individuals_weights=None):
        """
        Create instance of PCA object.
        
        Parameters
        ----------
        principal_components_number : int or None. Default to None.
        
            Number of principal components desired at the end of the PCA. 
            If None, all the principal components computed in numpy.linalg.svd are kept.
            If int, we keep the number entered as long as it is greater than the number 
            of variables or the number of individuals minus one.
  
        std_scale : bool
        
            Boolean to scale each columns (variable) by its standard deviation.
       
        features_weights : numpy array-like, list, str or None. Default to None.
  
            If numpy array-like or list, the coefficients are used as features weights.
            Must has same length than number of variables given when 'fit' function is used.

            If string, the string should represent a known method to compute the weights.
            The only method defined today is 'var', which weights each individual by the 
            inverse of the standard deviation of each variable (to reduce the importance of 
            variables with too much variability).

            If None, each variables has the same importance. The distance between 
            two individuals is then calculated with the Euclidean norm of the difference 
            in coordinates.
  
        individuals_weights : numpy array-like, list, or None. Default to None.
  
            If numpy array-like or list, the coefficients are used as individuals weights.
            Must has same length than number of individuals when 'fit' function is used.

            If None, each individual is weighted with the same weight. 
       """
        
        #################
        # save attributes
        #################
        
        self.do_std_scale = std_scale
        self.features_weights = features_weights
        self.individuals_weights = individuals_weights
        self.ncp = principal_components_number
        
        ###################################################
        # define types of qual / quant / temporal variables
        ###################################################
        
        self.quant_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        self.qual_dtypes = ['category']
        # self.temporal_dtypes = ['<M8[ns]']
        
        ###########################################
        # creation of attributes used in fit method
        ###########################################
        
        self.data = None
        self.X = None
        self.n = None
        self.p = None
        self.principal_components_inertias = None
        self.per_of_principal_comp_inertias = None
        self.cumulative_per_of_principal_comp_inertias = None
        self.individuals_coordinates = None
        self.features_coordinates = None
     
        
    def fit(self, data):
        """
        Compute the principals components and linked attributes with input data.
        
        Parameters
        ----------
        data : pandas dataframe
        
            Input data compound of quantitative and qualitative variables
            used to compute PCA decomposition.
            
            Warning : the data must always be in this form: individuals in row, 
            variables / features in column.
       """
        
        # if input is not a dataframe, raise error
        if not isinstance(data, pd.DataFrame):
            raise TypeError("input data is not pandas dataframe")
        else:
            self.data = data

        # if other input than quantitative or quantitative, warn message
        if len(data.select_dtypes(exclude = self.quant_dtypes + self.qual_dtypes).columns) != 0:
            raise TypeError("some dataframe columns are not quantitative or qualitative variable")
            
        self.X = data.select_dtypes(include = self.quant_dtypes).to_numpy()
        self.n = self.X.shape[0]
        self.p = self.X.shape[1]
        self.update_ncp()
 
        self.X -= np.mean(self.X, axis=0)
            
        if self.do_std_scale:
            self.X /= np.std(self.X, axis=0)
        
        U, S, V = self.SVD(self.X, self.get_individuals_weights(), self.get_features_weights(), self.ncp)
        
        # inertias are eigen vectors of covariance matrix, 
        # so square of singular values 
        self.principal_components_inertias = np.square(S)
        
        # computation of percentages
        self.per_of_principal_comp_inertias = (self.principal_components_inertias / np.sum(self.principal_components_inertias)) * 100.0
        self.cumulative_per_of_principal_comp_inertias = np.cumsum(self.per_of_principal_comp_inertias)
    
        # update of number of principal components
        U = U[:, :self.ncp]
        S = S[:self.ncp]
        V = V[:, :self.ncp]
    
        # computation of individuals and features coordinates in 
        # principal components space
        self.individuals_coordinates = U * np.abs(S)
        if self.do_std_scale:
            self.features_coordinates = V * np.abs(S)
        else:
            self.features_coordinates = V * (np.abs(S) / np.sqrt(np.abs(S)))
        
    def get_features_weights(self):
        # if None, all features are equivalently weighted
        if self.features_weights is None:
            return np.repeat(1, self.p)
        # if features weights indicate "var" method, return identity matrix weighted by each 
        # input numerical variable variance
        elif self.features_weights == "var":
            return np.reciprocal(np.var(self.X, axis=0))
        
        # if list of weights as input, we check the length and return it
        elif isinstance(self.features_weights, list) or isinstance(self.features_weights, np.ndarray):
            if (len(np.shape(self.features_weights)) == 1) and (len(self.features_weights) == self.p):
                return self.features_weights
            # error if length of input list is not p
            else:
                raise ValueError("features_weights attribute is list of shape {} instead of ({},)".format(np.shape(self.features_weights), self.p))
        # error if other object is given as input
        else:
            raise ValueError('unkwnown features_weights attribute')
            
    def get_individuals_weights(self):
        # if None, all individuals are equivalently weighted
        if self.individuals_weights is None:
             return np.repeat(1.0/float(self.n), self.n)
        # if list of weights as input, we check the length and return it
        elif isinstance(self.individuals_weights, list) or isinstance(self.individuals_weights, np.ndarray):
            if (len(np.shape(self.individuals_weights)) == 1) and (len(self.individuals_weights) == self.n):
                return self.individuals_weights
            # error if length of input list is not n
            else:
                raise ValueError("individuals_weights attribute is list of shape {} instead of (,{})".format(np.shape(self.individuals_weights), self.n))
        # error if other object is given as input
        else:
            raise ValueError('unkwnown individuals_weights attribute')
            
    def update_ncp(self):
        if self.ncp is None:
            self.ncp = min(self.n - 1, self.p)
        elif (self.ncp > self.n - 1) or (self.ncp > self.p):
            warning = "{} desired principal components".format(self.ncp)
            self.ncp = min(self.n - 1, self.p)
            warnings.warn("{} but n = {} individuals and p = {} features --> {} principal components kept".format(warning, self.n, self.p, self.ncp))
            
    @staticmethod
    def SVD(X, individual_weights = None, variable_weights = None, n=np.inf):

        # definition of individual_weights if None
        if individual_weights is None:
            ind_weights = np.repeat(1.0/float(X.shape[0]), X.shape[0])
        else:
            ind_weights = individual_weights

        # definition of variable_weights if None
        if variable_weights is None:
            var_weights = np.repeat(1, X.shape[1])
        else:
            var_weights = variable_weights

        ind_weights /= np.sum(ind_weights)

        ncp = np.min(np.array([n,X.shape[0]-1,X.shape[1]]))

        # ponderation of x by ind and var weights
        x = X * np.sqrt(var_weights)
        x = (x.T * np.sqrt(ind_weights)).T    

        U, S, VH = np.linalg.svd(x, full_matrices=False)
        V = VH.T

        if ncp > 1:
            mult = np.sign(np.matmul(np.repeat(1, V.shape[0]).T, V))
            mult[mult == 0] = 1
            mult = mult.reshape((mult.shape[0], 1))
            U = (U.T * mult).T
            V = (V.T * mult).T

        ind_weights = ind_weights.reshape((ind_weights.shape[0], 1))
        var_weights = var_weights.reshape((var_weights.shape[0], 1))

        U = np.divide(U, np.sqrt(ind_weights))
        V  = np.divide(V, np.sqrt(var_weights))

        return U, S, V
    
    def components_barplot(self, n_components:int=None, mode:int=0, bar_color=None):
        """
        Return plotly barplot of the inertias of principals components.
        
        Parameters
        ----------
        n_components : int. Default to None.
        
            Number of computed principal components to plot.
            
        mode : int. Default to 0.
        
            If 0, return a barplot of relative inertias. 
            If 1, return a cumulative sum of relative inertias.
            
        bar_color : str (or plotly.graph_objs._figure.Figure accepted CSS colors), None, int or float.
                    Default to None.
        
            If str or plotly accepted colors, will color the bars with this color.
            If None, will color the bars with "DarkSeaGreen" CSS color.
            
            If int or float, the parameter represents a minimum inertia and will be used as a coloring rule. 
            The principal component bars that provide at least the inertia that the parameter represents will 
            be colored in "DarkSeaGreen" CSS color, the others in "IndianRed" CSS color. 
       """
        
        # if n_components is None, we plot as many principal components inertia as keeped in last fit method
        if n_components is None:
            n = self.ncp
        else:
            # if n_components is not None and is greater than number of keep principal components
            # we only plot the keeped principal components inertias and warn the user
            if n_components > self.ncp:
                warnings.warn("{} inertia components requested while {} available, the number of requested inertia components has been truncated".format(n_components, self.ncp))
                n = self.ncp
            # if n_components lesser than keeped in last fit method, we only plot the highest n_components components inertias
            else:
                n = n_components
        
        # change of plot parameters and input data if absolute or cumulative sum inertias is wanted
        if mode == 0:
            y = self.per_of_principal_comp_inertias[:n]
            title = "Percentages of inertias of PCA components"
            yaxis_title = "Percentage of inertia"
        elif mode == 1:
            y = self.cumulative_per_of_principal_comp_inertias[:n]
            title = "Cumulative percentages of inertias of PCA components"
            yaxis_title = "Cumulative percentage of inertia"
        else:
            raise ValueError("mode must be 0 (for relative inertia barplot) or 1 (for cumulative relative inertia barplot")

        # labels for each bar in the bar plot ("Component 1", "Component 2", etc)
        labels = np.char.add(np.repeat("Component ", n), np.arange(1, n+1).astype("str"))
        
        # compute of the barplot
        fig = px.bar(x=labels, y=y)
        
        # if no bar color provided, default to DearkSeaGreen
        if bar_color is None:
            color="DarkSeaGreen"
        # if float or int, we color in green the bar that provide at least the wanted inertia
        elif isinstance(bar_color, int) or isinstance(bar_color, float):
            keeped_inertias = self.cumulative_per_of_principal_comp_inertias[self.cumulative_per_of_principal_comp_inertias < bar_color]
            if len(keeped_inertias) == n:
                color="DarkSeaGreen"
            else:
                n_green = len(keeped_inertias)+1
                n_red = n - n_green
                color = np.concatenate((np.repeat("DarkSeaGreen", n_green), np.repeat("IndianRed", n_red)))
        # if plotly-accepted color provided, plot the bars with this color
        else:
            color=bar_color

        # update of bar colors and axis titles, inertia range, etc
        fig.update_traces(marker=dict(color=color))
        fig.update_layout(template="gridon", title=title, 
                  xaxis_title="PCA components number", 
                  yaxis_title=yaxis_title,
                  yaxis_range=[0.0,110.0])
        
        return fig
    
    def individuals_factor_map(self, x:int=1, y:int=2, z=None, label=None, marker_size:int=8, marker_color=None):
        """
        Return plotly scatter plot of the projected individuals on principal components.

        Parameters
        ----------
        x : int. Default to 1. 

            Which principal component to choose as abscissa.
            Must be lesser than keeped number of components in last fit method.

        y : int. Default to 2.

            Which principal component to choose as ordinate.
            Must be lesser than keeped number of components in last fit method.

        z : None or object. Default to None.

            Label of column representing a categorical variable of input data 
            during the last run of fit method.

        label : None or object. Default to None.

            Label of column representing a categorical variable used to 
            display hover labels on each individuals plotted in scatter plot.

        marker_color : str (or plotly.graph_objs._figure.Figure accepted CSS colors) or None.
                       Default to None.

            Str or plotly accepted colors to color the markers of the scatter plot.
        """

        # check that the wanted principal components exists
        if x > self.ncp :
            raise ValueError("only {} components available while the {}nth is requested".format(self.ncp, x))
        elif y > self.ncp :
            raise ValueError("only {} components available while the {}nth is requested".format(self.ncp, y))

        if z is not None:
            color = self.data[z]
        else:
            color = None

        if label is not None:
            hover_name = self.data[label]
        else:
            hover_name = None

        fig = px.scatter(x=self.individuals_coordinates[:,x-1], y=self.individuals_coordinates[:,y-1], color=color, hover_name=hover_name)

        # update of title, axis title, etc
        fig.update_layout(template="gridon", title="Individuals factor map for components {} and {}".format(x, y), 
                          xaxis_title="Component {} ({}%)".format(x, round(self.per_of_principal_comp_inertias[x-1], 2)), 
                          yaxis_title="Component {} ({}%)".format(y, round(self.per_of_principal_comp_inertias[y-1], 2)))

        # if no categorical colors, we can use scatter marker color with marker_color parameters
        if z is None:
            # default to IndianRed, else use parameter
            if marker_color is None:
                 fig.update_traces(marker=dict(size=marker_size, color="IndianRed"))
            else:
                 fig.update_traces(marker=dict(size=marker_size, color=marker_color))
        # if z is not None
        else:
            # marker_color won't have any effect
            if marker_color is not None:
                warnings.warn("'{}' specified as marker color not taken into account because z_color is specified".format(marker_color))
            fig.update_traces(marker=dict(size=marker_size))
            fig.update_layout(legend_title_text=z)

        return fig
    
    def variables_factor_map(self, x:int=1, y:int=2, size:int=400, font_size:int=8):
        """
        Return plotly circle plot of variables projected (coords are correlation) on principal components axes.
        
        Parameters
        ----------
        x : int. Default to 1. 
        
            Which principal component to choose as abscissa.
            Must be lesser than keeped number of components in last fit method.
            
        y : int. Default to 2.
        
            Which principal component to choose as ordinate.
            Must be lesser than keeped number of components in last fit method.
            
        size : int. Default to 400.
                       
            Size of the plot.
            
        font_size : int. Default to 8.
        
            Font size of used in the plot.
       """
        
        # check that the wanted principal components exists
        if x > self.ncp :
            raise ValueError("only {} components available while the {}nth is requested".format(self.ncp, x))
        elif y > self.ncp :
            raise ValueError("only {} components available while the {}nth is requested".format(self.ncp, y))
        
        # Create a dataframe with x and y coordinate for each label
        coords = pd.DataFrame({"Feature" : self.data.select_dtypes(include = self.quant_dtypes).columns.to_list(),
            "Component {}".format(x) : self.features_coordinates[:,x-1], "Component {}".format(y) : self.features_coordinates[:,y-1], 
                               })
                
        # add (0.0,0.0) coordinates to each features to plot the lines from the centers
        centers = coords.copy(deep=True)
        centers.iloc[:,[1, 2]] = 0.0
        coords = pd.concat([coords, centers]).sort_values("Feature")
        
        # create the plot
        fig = px.line(data_frame=coords, x=coords.iloc[:,1].name, y=coords.iloc[:,2].name, color="Feature", width=size+75, height=size)
        
        # update titles, etc        
        fig.update_layout(template="gridon", title="Variables factor map for components {} and {}".format(x, y),
                          xaxis_title="Component {} ({}%)".format(x, round(self.per_of_principal_comp_inertias[x-1], 2)),
                          yaxis_title="Component {} ({}%)".format(y, round(self.per_of_principal_comp_inertias[y-1], 2)),
                          yaxis_range=[-1.2,1.2],
                         xaxis_range=[-1.2, 1.2],
                         font=dict(size=font_size),
                         showlegend=True)
        fig.update_traces(mode='lines+markers')
        
        # add the circle
        fig.add_shape(type="circle", x0=-1.0, y0=-1.0, x1=1.0, y1=1.0, line_color="Black", line_width=1.0)
        
        return fig

In [35]:
X = [[6, 6, 5, 5.5], [8, 8, 8, 8], [6, 7, 11, 9.5], [14.5, 14.5, 15.5, 15], [14, 14, 12, 12.5], [11, 10, 5.5, 7], [5.5, 7, 14, 11.5], [13, 12.5, 8.5, 9.5], [9, 9.5, 12.5, 12]]
X = pd.DataFrame(X, columns=['Maths', 'Physique', 'Français', 'Anglais'], index=['Nathan', 'Emma', 'Lola', 'Baptiste', 'Mathilde', 'Ines', 'Lucas', 'Aymeric', 'Chloe'])
X.reset_index(drop=False, inplace=True)
X.rename(columns={"index": "Prénom"}, inplace=True)
X['Prénom'] = X['Prénom'].astype('category')

acp = PCA(std_scale=False)
acp.fit(X)
acp.variables_factor_map()